In [79]:
# generic imports
import sys
import os
sys.path.append('../')
import img_processing_util
from definitions import *

import seaborn as sns
from tqdm import tqdm
import json
import pandas as pd
import pickle
from collections import defaultdict
import math
import numpy as np
#import krippendorff
import matplotlib.image as mpimg
from datetime import datetime
#stats amd ml
from scipy import stats
import matplotlib.pyplot as plt
import cv2
import csv


current_dataset = "coco"
current_dataset = "google"

openImg_path = '/nfs/juhu/data/rakhasan/bystander-detection/google-img-db/'
coco_path = '/nfs/juhu/data/rakhasan/bystander-detection/coco/'
if current_dataset=="google":
    survey_path='/nfs/juhu/data/rakhasan/bystander-detection/pilot-study2/'
else:
    survey_path='/nfs/juhu/data/rakhasan/bystander-detection/pilot3_coco/'

mapping_path = survey_path+'mappings.pkl'
common_mapping_path = survey_path+'mappings_common.pkl'
survey_photo_path = survey_path+'/photos/'
cropped_photo_path = survey_path+'/cropped-photos/'
IMG_SIZE = (256, 256)

In [73]:
if current_dataset=="google":
    human_parts,object_classes =pickle.load(open(survey_path+'human-object-classes.pkl','rb'))
    survey_photo_annotations =pickle.load(open(survey_path+'survey_photo_annotations.pkl','rb'))
    body_features = ['_'.join(human_parts[body_part].split()) for body_part in human_parts.keys()]

In [81]:
body_features=['Human_eye',
 'Human_beard',
 'Human_mouth',
 'Human_body',
 'Human_foot',
 'Human_leg',
 'Human_ear',
 'Human_hair',
 'Human_head',
 'Human_arm',
 'Human_nose',
 'Human_hand']

mapping = pickle.load(open(common_mapping_path,'rb'))
photo_df = pd.read_pickle(os.path.join(survey_path, 'photo_df.pkl'))
photo_df.index=photo_df.index.astype(int)
photo_df.shape

(15488, 26)

In [82]:
mapping[0]

{'bbox': [272.943875, 392.698627, 530.510005, 1083.8161169999998],
 'fileName': '03e3ee823d39aa67.jpg',
 'imageId': '03e3ee823d39aa67',
 'index': 0}

In [75]:
'''
    Common mapping format:
        imgId: original image id
        index: index in the mapping 
        fileName: original image file name
        bbox: [xmin, xmax, ymin, ymax]     
'''

# '''change coco-mapping to common format'''
# new_mapping = []
# for i in range(len(mapping)):
#     m=mapping[i]
#     box=m['bbox']
#     d={
#         'imageId': m['image_id'],
#         'index': i,
#         'fileName':'COCO_train2014_000000{}.jpg'.format(m['image_id']),
#         'bbox':[box[0],box[0]+box[2],box[1],box[1]+box[3]]
#     }           
#     new_mapping.append(d)
# pickle.dump(new_mapping, open(survey_path+'mappings_common.pkl','wb'))
# json.dump(new_mapping, open(survey_path+'mappings_common.json','w'))


# '''change google-mapping to common format'''
# new_mapping = []
# for i in tqdm(range(len(mapping))):
#     m=mapping[i]
#     img = cv2.imread(survey_photo_path+str(i)+'.jpg')
#     r=m[4:8]
#     box=[float(r[0])*len(img[0]), 
#          float(r[1])*len(img[0]),
#          float(r[2])*len(img),
#          float(r[3])*len(img)]
#     d={
#         'imageId': m[0],
#         'index': i,
#         'fileName':'{}.jpg'.format(m[0]),
#         'bbox':[box[0],box[0]+box[2],box[1],box[1]+box[3]]
#     }           
#     new_mapping.append(d)
#pickle.dump(new_mapping, open(survey_path+'mappings_common.pkl','wb'))
#json.dump(new_mapping, open(survey_path+'mappings_common.json','w'))

'\n    Common mapping format:\n        imgId: original image id\n        index: index in the mapping \n        fileName: original image file name\n        bbox: [xmin, xmax, ymin, ymax]     \n'

In [76]:
#high_concept_data = photo_df[high_level_concepts_num+['subject_bystander_num','contains_full_body']]
high_concept_data = photo_df[high_level_concepts_num+['subject_bystander_num']]
'''Remove NaN rows'''
print(high_concept_data.shape)
high_concept_data =high_concept_data[~high_concept_data.isnull().any(axis=1)]
high_concept_data = high_concept_data.astype(float)
print(high_concept_data.shape)

(15488, 8)
(14803, 8)


In [59]:
feature_df = pd.DataFrame()

# add numerical features and class label directly obtained from the survey
for c in high_level_concepts_num+['subject_bystander_num']:#,'contains_full_body']:
    feature_df[c] = high_concept_data.groupby(high_concept_data.index).mean()[c]

def get_class_label(val):
    '''Return 1, -1, or 0 for subject, bystander, and neither'''
    if val > 0:
        return 1
    if val < 0:
        return -1
    return 0

feature_df['label'] = feature_df.apply(lambda r: get_class_label(r.subject_bystander_num), axis=1)

#assert(len(set(high_concept_data.index.values)) == len(feature_df))
print('Total unique photos: ',len(feature_df))

feature_df.reset_index(inplace=True)
feature_df['photo_no']=feature_df.photo_no.astype(int)
feature_df.set_index('photo_no', inplace=True)
feature_df.head()

Total unique photos:  4080


,was_aware_num,posing_num,comfort_num,will_num,photographer_intention_num,replacable_num,photo_place_num,subject_bystander_num,label
photo_no,,,,,,,,,
1,2.50,2.00,2.00,1.50,2.5,-2.50,2.00,1.75,1
2,2.25,2.25,2.00,1.75,2.0,-0.25,0.00,0.75,1
3,-2.00,0.00,0.25,0.50,-2.0,2.75,-0.50,-1.75,-1
4,-2.00,-2.00,0.25,0.50,-1.0,2.75,0.00,-1.25,-1
5,1.25,0.00,1.25,1.00,-1.5,0.50,-1.75,-1.25,-1


In [224]:
'''
Create a reverse map where key=imgeID and values are the list of photono used in the survey.
The length of the list indicates how many actual people are in the photo.
'''
reverse_map = defaultdict(list)
photo_numbers = set(feature_df.index.values.astype(int))
for m in range(len(mapping)):
    if m in photo_numbers: # if that is a real person
        reverse_map[mapping[m]['imageId']].append(m)
        
'''Create another dict where keys are number of people, and values are list of imgeIds containing that many people'''
photo_by_people = defaultdict(list)
for rm in reverse_map:
    photo_by_people[len(reverse_map[rm])].append(rm)
    
#pickle.dump(reverse_map, open(os.path.join(survey_path, 'reverse-map.pkl'), 'wb'))
#pickle.dump(photo_by_people, open(os.path.join(survey_path, 'photo-by-people.pkl'), 'wb'))
reverse_map= pickle.load(open(os.path.join(survey_path, 'reverse-map.pkl'), 'rb'))

In [225]:
# feature_df[feature_df.contains_full_body<.51].groupby(['contains_full_body','label']).subject_bystander_num.count()
# #feature_df[['contains_full_body','label']]
# feature_df[feature_df.contains_full_body<.51].groupby(['label']).subject_bystander_num.count()
# feature_df=feature_df[feature_df.contains_full_body>.5]
# feature_df.shape

In [77]:
mapping[0]

{'bbox': [272.943875, 392.698627, 530.510005, 1083.8161169999998],
 'fileName': '03e3ee823d39aa67.jpg',
 'imageId': '03e3ee823d39aa67',
 'index': 0}

In [ ]:
'''Add image level features'''
def get_person_size(img, r):
    '''
    Returns size of a person (both absolute and percentage) in a photo given the photo or its path and 
    rectangle 'r' containing the person.
    '''
    if isinstance(img, str):
        #print(img)
        img= mpimg.imread(img)
        
    img_w, img_h = len(img[0]), len(img)
    xmin, xmax, ymin, ymax = r[0],r[1],r[2],r[3]
    
    size = (float(xmax)-float(xmin))* (float(ymax)-float(ymin))
    ret=(size, size/(img_w * img_h))
    #print(ret)
    return ret


# feature_df['person_distance_axes_norm'] = feature_df.apply(lambda row: 
#         helper.get_distance_from_center( img = survey_photo_path+str(row.name)+'.jpg',
#                                          r= mapping[int(row.name)][4:8],
#                                          center_zero= False, norm_type='Axis_size', draw=False)[0], axis=1)
# feature_df['person_distance_img_norm'] = feature_df.apply(lambda row: 
#         helper.get_distance_from_center( img = survey_photo_path+str(row.name)+'.jpg',
#                                          r= mapping[int(row.name)][4:8],
#                                          center_zero= False, norm_type='Img_size', draw=False)[0], axis=1)

# feature_df['person_distance']=feature_df.person_distance_axes_norm
feature_df['person_size'] = feature_df.apply(lambda row: 
        get_person_size(
            img = survey_photo_path+str(row.name)+'.jpg', 
            r=   mapping[int(row.name)]['bbox'])[1], axis=1)#only keep relative size



In [62]:
#feature_df['num_people'] = feature_df.apply(lambda row: len(reverse_map[mapping[int(row.name)]['imageId']]), axis=1)

'''Save only highlevel features'''
binary_feature_df = feature_df[(feature_df.label==1)|(feature_df.label==-1)]
print('Before:',len(feature_df),'Now:',len(binary_feature_df))

print("Any null: ",binary_feature_df.isnull().values.any())
binary_feature_df['label']=binary_feature_df.apply(lambda row: 1 if row.label==1 else 0, axis=1)

'''Check data distribution'''

print('\n\nPos:{}({:.2f}%)\nNeg:{}({:.2f}%)'.format(
    len(binary_feature_df[binary_feature_df.label==1]), len(binary_feature_df[binary_feature_df.label==1])*100/len(binary_feature_df),
     len(binary_feature_df[binary_feature_df.label==0]), len(binary_feature_df[binary_feature_df.label==0])*100/len(binary_feature_df)))

'''Save features'''
binary_feature_df.to_pickle(os.path.join(survey_path, 'high-feature-df.pkl'))
binary_feature_df.to_csv(os.path.join(survey_path, 'high-feature-df.csv'))

Before: 4080 Now: 3802
Any null:  False


Pos:2287(60.15%)
Neg:1515(39.85%)


/l/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [85]:
feature_df[['person_size','subject_bystander_num']].corr()

,person_size,subject_bystander_num
person_size,1.000000,-0.201247
subject_bystander_num,-0.201247,1.000000


In [64]:
binary_feature_df[['person_size','label']].corr()

,person_size,label
person_size,1.000000,-0.190423
label,-0.190423,1.000000


In [228]:
'''Which human body parts and objects are annotated in the photos'''

# def contains_body_part(photo_index, body_part, person_annotation, annotation_dict):
#     out = 0
#     try:
#         annotations = annotation_dict[mapping[photo_index][0]]
#         for anno in annotations:
#             if body_part == anno[2]: #object id in 3rd position of annotation 
#                 if helper.rect_in_rect(anno[4:8], person_annotation[4:8], overlap_thresh = 70):
#                     out+=1
#     except Exception as e:
#         print(photo_index,e)
#     return out
    
# for body_part in human_parts.keys():
#     feature_df['_'.join(human_parts[body_part].split())] = feature_df.apply(lambda row: 
#         contains_body_part(int(row.name), body_part, mapping[int(row.name)], survey_photo_annotations), axis=1)

'Which human body parts and objects are annotated in the photos'

In [229]:
#feature_df.groupby('label').mean()[body_features]

In [230]:
'''Include openpose features: joint probability and angle'''
openpose_feat = pd.read_csv(survey_path+'/openpose-feature.csv')
openpose_feat.set_index('photo_no', inplace=True)
print(openpose_feat.shape)
#openpose_feat.fillna(0, inplace=True) ## remove NaNs
for c in openpose_feat.columns:
    feature_df[c] = openpose_feat[c]

(749, 35)


In [231]:
'''Include resnet features'''
resnet_feat = pd.read_csv(survey_path+'/resnet-features-df.csv')
resnet_feat.set_index('photo_no', inplace=True)
for c in ['resnet_feat_avg_'+str(i) for i in range(2048)]:
    feature_df[c]=resnet_feat[c]

In [232]:
'''Include facial expression'''
expressions = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']
#exp_dict = pickle.load(open(survey_path+'facial-expressions.pkl','rb'))
exp_df = pd.read_csv(survey_path+'/face-exp-feature.csv')#pd.DataFrame.from_dict(exp_dict).T
exp_df.set_index('photo_no', inplace=True)
exp_df[exp_df < 0] = 0

for e in expressions:
    feature_df[e] = exp_df[e]

In [233]:
feature_df.shape

(784, 2101)

In [234]:
'''Remove neither label'''
binary_feature_df = feature_df[(feature_df.label==1)|(feature_df.label==-1)]
print('Before:',len(feature_df),'Now:',len(binary_feature_df))

print("Any null: ",binary_feature_df.isnull().values.any())
binary_feature_df['label']=binary_feature_df.apply(lambda row: 1 if row.label==1 else 0, axis=1)

'''Check data distribution'''

print('\n\nPos:{}({:.2f}%)\nNeg:{}({:.2f}%)'.format(
    len(binary_feature_df[binary_feature_df.label==1]), len(binary_feature_df[binary_feature_df.label==1])*100/len(binary_feature_df),
     len(binary_feature_df[binary_feature_df.label==0]), len(binary_feature_df[binary_feature_df.label==0])*100/len(binary_feature_df)))

'''Save features'''
binary_feature_df.to_pickle(os.path.join(survey_path, 'all-features-df.pkl'))
binary_feature_df.to_csv(os.path.join(survey_path, 'all-features-df.csv'))

Before: 784 Now: 736
Any null:  False


Pos:409(55.57%)
Neg:327(44.43%)


/l/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Inter-rater agreement among themselves

In [20]:
# labeled_photo_df = photo_df.loc[feature_df.index]
# print(len(labeled_photo_df.index),len(set(labeled_photo_df.index)), len(feature_df))
# labeled_photo_df=labeled_photo_df[~np.isnan(labeled_photo_df.subject_bystander_num)]
# print(len(labeled_photo_df.index),len(set(labeled_photo_df.index)), len(feature_df))
# labeled_photo_df

In [22]:
# alphas=[]
# kappas=[]
# for i in range(100):
#     '''Create matrix'''
#     #print(i)
#     idx = range(i*50, (i+1)*50)
#     df=labeled_photo_df.loc[list(idx)][['label','photo_no']].dropna()#.groupby('photo_no').count()
#     print(i*50,(i+1)*50,df.shape)
#     mat = []
#     for p in set(df.index):
#         labels = list(df.loc[p].label)
#         d=dict()
#         for c in range(len(labels)):
#             d['coder'+str(c)]=labels[c]
#         mat.append(d)
#     #'''Calculate agreement'''
#     code_df = pd.DataFrame(mat).T
#     alphas.append(krippendorff.alpha(code_df))
#     #cohen_kappa_score(labeler1, labeler2)

### Inter-rater agreement with the final label

In [153]:
'''Check how much agreement there is among the raters'''
def agreement_perc(photo_no,label):
    rates = photo_df.loc[photo_no]
#     if  isinstance(rates, pd.core.series.Series):
#         return 0
    same_rate = rates[rates.subject_bystander_num*label>0]
    #print(len(same_rate),len(rates))
    return int(100*len(same_rate)/len(rates))
    
agreement_df = pd.DataFrame()
agreement_df['label']=feature_df[feature_df.label != 0].label
agreement_df['agreement'] = agreement_df.apply(lambda row: agreement_perc(row.name, row.label), axis=1)
#agreement_df['disagreement'] = agreement_df.apply(lambda row: agreement_perc(row.name, 1-row.label), axis=1)

pickle.dump(list(agreement_df[(agreement_df.agreement<=65)].index),open(survey_path+'low-agreement.pkl','wb'))
pickle.dump(list(agreement_df[(agreement_df.agreement>65)&(agreement_df.agreement<67)].index),open(survey_path+'agreement23.pkl','wb'))
pickle.dump(list(agreement_df[(agreement_df.agreement>=100)].index),open(survey_path+'agreement100.pkl','wb'))

count = agreement_df.groupby(['agreement']).size()
print(np.mean(agreement_df.agreement))
count[count>100],

84.67862266857962


(agreement
 66     197
 100    419
 dtype: int64,)

In [146]:
fourth=pd.read_pickle(os.path.join(survey_path, 'fourth_df.pkl'))
fourth.index=fourth.index.astype(int)

remaining=set(agreement_df[agreement_df.agreement==66].index).difference(set(fourth.index))
print(len(remaining))
list(remaining)

118


[834,
 121,
 122,
 123,
 131,
 650,
 139,
 651,
 652,
 655,
 656,
 660,
 661,
 663,
 664,
 665,
 666,
 667,
 671,
 673,
 674,
 675,
 677,
 684,
 686,
 690,
 691,
 696,
 697,
 705,
 710,
 716,
 207,
 209,
 210,
 723,
 214,
 215,
 729,
 218,
 219,
 732,
 224,
 227,
 228,
 739,
 234,
 235,
 236,
 241,
 243,
 245,
 247,
 249,
 802,
 803,
 805,
 807,
 306,
 818,
 308,
 819,
 310,
 311,
 312,
 313,
 827,
 316,
 317,
 318,
 832,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 838,
 833,
 330,
 331,
 332,
 333,
 844,
 335,
 845,
 337,
 851,
 341,
 854,
 856,
 345,
 346,
 347,
 864,
 865,
 869,
 874,
 881,
 882,
 400,
 404,
 406,
 407,
 410,
 412,
 413,
 415,
 423,
 433,
 440,
 444,
 445,
 446,
 505,
 510,
 511]

In [171]:
'''Compare with fourth rater for 67% agreement'''
fourth=pd.read_pickle(os.path.join(survey_path, 'fourth_df.pkl'))
fourth.index=fourth.index.astype(int)
print(len(set(fourth.index).intersection(set(agreement_df[agreement_df.agreement==66].index))))
fourth=fourth.loc[list(set(fourth.index).intersection(set(agreement_df[agreement_df.agreement==66].index)))]
#fourth=fourth.loc[list(set(fourth.index).intersection(set(binary_feature_df.index)))]
fourth_binary=fourth[fourth.subject_bystander_num!=0]
print(len(fourth),len(fourth_binary))
fourth_binary['label']=fourth_binary.subject_bystander_num.clip(0,1)
fourth_binary['label2']=binary_feature_df.label
fourth_binary['agreed']= fourth_binary.label==fourth_binary.label2
#fourth_binary.apply(lambda row: 
 #   True if int(row.label)==int(binary_feature_df.loc[int(row.name)].label) else False, axis=1)
len(fourth_binary[fourth_binary.agreed])/len(fourth_binary)

170
170 152


/l/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/l/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/l/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

0.6973684210526315

In [159]:

binary_feature_df.head()#.loc[[0,1,2]]

,was_aware_num,posing_num,comfort_num,will_num,photographer_intention_num,replacable_num,photo_place_num,subject_bystander_num,label,person_size,num_people
photo_no,,,,,,,,,,,
0,-1.000000,-0.666667,0.666667,0.000000,1.000000,2.666667,-1.000000,-0.666667,0,0.004914,0
1,2.666667,2.000000,1.666667,1.333333,2.333333,-0.333333,2.000000,0.666667,1,0.564679,1
2,2.333333,2.333333,2.333333,1.666667,2.333333,0.000000,-0.666667,1.000000,1,0.083390,1
4,-0.500000,0.000000,0.000000,0.500000,-1.500000,2.500000,0.000000,-2.000000,0,0.004002,1
6,0.000000,-1.000000,0.500000,1.000000,0.500000,0.500000,-0.500000,-0.500000,0,0.069712,1


In [60]:
'''Correlation between size and subject'''
set(binary_feature_df.label)
stats.spearmanr(binary_feature_df.num_people, binary_feature_df.label)

NameError: name 'binary_feature_df' is not defined

In [ ]:
binary_feature_df.groupby(['num_people', 'label']).label.count()

In [ ]:
# for c in high_level_concepts_num:
#     print(c)
#     sns.distplot(feature_df[c])
#     plt.show()

In [ ]:
sns.distplot(feature_df.photo_place_num)

In [ ]:
len(feature_df[feature_df.photo_place_num<0])/len(feature_df)

In [ ]:
len(feature_df[feature_df.photo_place_num>0])/len(feature_df)

In [ ]:
public_place=feature_df[feature_df.photo_place_num>0]#
private_place=feature_df[feature_df.photo_place_num<0]
len(public_place[public_place.subject_bystander_num>0])/len(public_place),len(private_place[private_place.subject_bystander_num>0])/len(private_place)

In [12]:
photo_df[['photographer_intention','photographer_intention_num']]

,photographer_intention,photographer_intention_num
photo_no,,
235,Agree,2.0
238,Strongly agree,3.0
233,Strongly agree,3.0
214,Strongly disagree,-3.0
213,Strongly disagree,-3.0
234,Strongly disagree,-3.0
217,Neither agree nor disagree,0.0
239,Strongly disagree,-3.0
231,Strongly agree,3.0


In [24]:
feature_df[['photographer_intention_num','label']].corr()

,photographer_intention_num,label
photographer_intention_num,1.000000,-0.286463
label,-0.286463,1.000000


In [ ]:
photo_df[photo_df.subject_bystander_num*photo_df.photographer_intention_num<0][['photographer_intention','photographer_intention_num','subject_bystander_num']].head(20).dropna()

In [53]:
#binary_data['photographer_intention']=photo_df.photographer_intention
binary_data['x']=binary_data.label*binary_data.photographer_intention_num>0
binary_data[binary_data.x==False][['photographer_intention_num','label']].head(20)


/l/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,photographer_intention_num,label
photo_no,,
11,0.500000,-1
13,1.750000,-1
14,2.000000,-1
22,0.750000,-1
27,0.750000,-1
28,1.500000,-1
30,0.500000,-1
43,1.250000,-1
45,2.000000,-1


In [36]:
len(binary_data[binary_data.x]),len(binary_data[binary_data.x==False])

(1312, 2490)

In [26]:
binary_data = feature_df[(feature_df.label==1) | (feature_df.label==-1)]
binary_data[['photographer_intention_num','label']].corr()

,photographer_intention_num,label
photographer_intention_num,1.000000,-0.290037
label,-0.290037,1.000000


In [27]:
binary_data.shape


(3802, 9)

In [28]:
feature_df.shape

(4080, 9)

In [38]:
photo_df.loc[[11]][['photographer_intention_num','subject_bystander_num']]

,photographer_intention_num,subject_bystander_num
photo_no,,
11,1.0,0.0
11,-3.0,-2.0
11,3.0,-2.0
11,1.0,-1.0
